# connie-skipper

Repository at [https://github.com/PhMota/connie-skipper](https://github.com/PhMota/connie-skipper)

In [1]:
%load_ext autoreload
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
# import gc
from glob import glob
import xarray as xr
import connie_skipper as sk
import traceback
import re, os, time
import warnings
warnings.simplefilter("ignore", category=xr.core.extensions.AccessorRegistrationWarning)

In [2]:
from IPython.display import display, IFrame, clear_output
from ipywidgets import interact, interactive_output
import ipywidgets as widgets

In [3]:
sortpattern = lambda p: int(re.search(r"([0-9]+)\.fz", p).groups()[0])
class App:
    def select_update(self, pattern):
        size_fmt = lambda file: str(int(os.stat(file).st_size/1e6))
        path_fmt = lambda file: file.split(pattern.split("*")[0])[-1]
        time_fmt = lambda file: time.strftime( "%Y-%m-%d %H:%M", time.localtime(os.stat(file).st_ctime) )  #time.strftime( , 
        fmt = lambda file: f'{time_fmt(file)} {path_fmt(file)} [{size_fmt(file)}M]'
        files = sorted( glob(pattern), key=time_fmt )[::-1] 
        self.select.options = [(".", None)] + [ ( fmt(file), file ) for file in files ]
        
    def __init__(self, rootpath):
        self.da = None
        self.output = widgets.Output()
        self.pattern = widgets.Text(
            value = rootpath,
            placeholder = 'pattern for files',
            description = "file pattern",
            layout = { "width": "90%" }
        )
        self.select = widgets.Select( 
            options = [],
            layout = {'width': '90%'},
            rows = 10,
        )
        self.select_update(self.pattern.value)
        self.pattern.on_submit(
            lambda change: self.select_update(self.pattern.value)
        )
        self.progress = widgets.IntProgress(
            value = 0, 
            min = 0, 
            max = 10, 
            description = 'loading'
        )
        self.progressPlot = widgets.IntProgress(
            value = 0, 
            min = 0, 
            max = 10, 
            description = 'plotting'
        )
        self.app = None
        with self.output:
            display(self.pattern, self.select, self.progress)
        display(self.output)
        self.select.observe( 
            lambda path: self.plot_from_path2(path.new) if path.new is not None else None,
            names = "value"
        )
        
        self.interactive_figure = None
        trim = widgets.Checkbox(
            description = 'trim', 
            value = True
        )
        demod = widgets.Checkbox(
            description = 'demod/osi', 
            value = True
        )
        statsProj = widgets.Dropdown( 
            options = ["median", "mean", "std", "mad", "min", "max"], 
            description = 'stats(proj)' 
        )
        median_row = widgets.Checkbox(
            description = 'median(row)',
            value = True
        )
        median_col = widgets.Checkbox(description='median(col)')
        spectrum = widgets.Checkbox(description='spectrum', value=True)
        chid = widgets.IntSlider(
            value = 0,
            min = 0,
            max = 1,
            description = f'chid',
            continuous_update = False
        )
        samp = widgets.IntSlider( 
            value = 0,
            min = 0, 
            max = 1,
            description = f'samp',
            continuous_update = False
        )
        stats = widgets.Dropdown( options=["none", "median", "mean", "std", "mad", "min", "max"], description='stats(samp)' )
        stats.value = "mean"
        statsRange = widgets.IntRangeSlider(
            value = [0,1],
            min = 0,
            max = 1,
            description = f"{stats.value}",
            continuous_update = False
        )
        energyFactor = widgets.IntSlider(
            value = 1,
            min = 1,
            max = 49,
            description = "percentile",
            continuous_update = False
        )
        self.log = widgets.HTML(
            value = "",
            description = "messages",
        )
#         stats.observe( lambda opt: setattr(statsRange, "description", f"{opt.new}(samp)"), names="value" )
        ui = widgets.VBox([ 
            trim, 
            demod, 
            statsProj, 
            median_row, 
            median_col, 
            spectrum, 
            chid, 
            samp, 
            stats, 
            statsRange,
            energyFactor,
            self.progressPlot,
        ])
        self.app = {
            "median_row": median_row, 
            "median_col": median_col, 
            "spectrum": spectrum,
            "trim": trim, 
            "demod": demod, 
            "statsProj": statsProj, 
            "chid": chid, 
            "samp": samp, 
            "stats": stats, 
            "stats_range": statsRange,
            "energy_percentile": energyFactor,
            "path": self.select
        }
        self.plot = interactive_output( 
            self.gen_plot, 
            self.app
        )
        with self.output:
            display( widgets.HBox([ui, self.plot]) )
            display( self.log )
        
        
    def plot_from_path2(self, path):

        self.log.value = ""
        self.progress.value, self.progress.description = 1, "opening file"
        self.da = sk.fits_to_DataArray( path )

        self.progress.value, self.progress.description = 5, "widgets"
        chid_max = len(self.da["chid"].data)-1
        samp_max = int(self.da["samp"].data[-1])

        widget = self.app["stats_range"]
        _ = widget._trait_notifiers
        widget._trait_notifiers = {}
        widget.min = 0
        widget.max = samp_max+1
        widget.value = [0 if widget.max <= 100 else 10, widget.max]
        widget._trait_notifiers = _
        
        widget = self.app["chid"]
        _ = widget._trait_notifiers
        widget._trait_notifiers = {}
        widget.max = chid_max
        widget.value = 2
        widget.description = f'chid[0─{chid_max}]'
        widget._trait_notifiers = _

        widget = self.app["samp"]
        _ = widget._trait_notifiers
        widget._trait_notifiers = {}
        widget.max = samp_max
        widget.description = f'samp[0─{samp_max}]'
        widget._trait_notifiers = _
        

        self.progress.value, self.progress.description = 10, "complete"
        
    def gen_plot(self, 
        median_row, 
        median_col, 
        spectrum,
        demod,
        trim, 
        chid, 
        samp, 
        stats, 
        stats_range,
        statsProj,
        energy_percentile,
        path
    ):
        if self.da is None:
            return
        self.log.value = f"{self.da.attrs['path'][0]}<br>"
        self.log.value += f"{self.da.attrs['name']}<br>"
#         self.log.value += ''.join( [ f"<b>{key}</b> {value}<br>" for key, value in self.da.attrs.items() ] )
        try:
            op = lambda _da: _da.skipper.isel(chid=chid)
            if stats == "none":
                stats_str = stats
                op = lambda _da, prevop=op: prevop(_da).skipper.isel(samp=samp)
            else:
                stats_str = f"{stats}({stats_range[0]}, {stats_range[1]})"
                op = lambda _da, prevop=op: prevop(_da).skipper.isel(samp=slice(*stats_range)).skipper.stats(dim = "samp", mode = stats)
            if trim:
                op = lambda _da, prevop=op: prevop(_da).skipper.trim()
            if demod:
                op = lambda _da, prevop=op: prevop(_da).skipper.demodulate("col")
            else:
                op = lambda _da, prevop=op: prevop(_da).skipper.centralize_os("col")
            suptitle = path.replace("_", r"\_")
            self.interactive_figure = op(self.da).skipper.plot_full( 
                robust = True, 
                fig = self.interactive_figure, 
#                 title = ,
                xproj = median_row,
                yproj = median_col,
                spectrum = spectrum,
                energy_percentile = energy_percentile,
                mode = statsProj,
                suptitle = fr"{suptitle}\\chid={chid} samp={samp} stats={stats_str}",
                progress_bar = (self.progressPlot, 0, 10),
                log = self.log
            )
        except:
            self.log.value += "<b style='color:red'>error in plot</b><br>"
            self.log.value += traceback.format_exc()
        finally:
            self.log.value += f"{self.da.attrs['name']}<br>"
"/share/storage2/connie/data/lta/testAngra/"
App("/share/storage2/connie/data/runs/101/*NSAMP400*.fz");

Output()

# Noise & DC evolution

In [6]:
evo = {
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img172.fz': {
        'os': [-0.5080138380012229, 59.73979229247933, 88280.33043211675],
        'oserr': [0.04116326425985355, 0.02990437864547487, 60.50929596124626],
        'ac': [-11.08621036452469, 65.05127097447632, 399.31450637432954, 0.059279021296615914, 194759.7899844511],
        'acerr': [0.12400566526014921, 0.09027183559827984, 0.5194678626248276, 0.0004664974458035732, 351.83897662366775],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img175.fz': {
        'os': [-0.12318985142775424, 59.6168900481668, 88043.70534711864],
        'oserr': [0.059869139894729084, 0.04295589070340928, 87.95480457723501],
        'ac': [-11.513545910725831, 65.01090593408946, 401.74290565250794, 0.058747231401083976, 195104.37010938284],
        'acerr': [0.11683673242964468, 0.08465667208715605, 0.4886995724965246, 0.0004377349776392954, 332.25460002976945],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img178.fz': {
        'os': [-0.09440888886118483, 59.29045329455836, 88009.55244719776],
        'oserr': [0.1724025283795448, 0.1235015582284527, 254.4573451104596],
        'ac': [-11.040277882118023, 65.24232117185493, 401.1111480258752, 0.057424620701946136, 196961.9946182767],
        'acerr': [0.2422194514918753, 0.1789919539227636, 1.2042593136773057, 0.0009277073981869929, 694.2162025696215]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img169.fz': {
        'os': [-0.20859081124243195, 59.235486757979864, 87861.28811193097],
        'oserr': [0.13668607098827856, 0.09900597141194142, 201.69864428856928],
        'ac': [-11.153530727171127, 65.20187682097273, 397.63122187993355, 0.059704829522168705, 195685.73310275516],
        'acerr': [0.1950924705883128, 0.141420443197749, 0.7860025414500738, 0.0007043108373317838, 553.1052394932109],  
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img166.fz': {
        'os': [-0.020175608858074307, 59.59943382552798, 87980.82580917062],
        'oserr': [0.06886388773660937, 0.04960142310011065, 101.00515205203381],
        'ac': [-10.521841586281463, 64.96438359102828, 399.52865393650154, 0.05901887142733882, 194963.21578329353],
        'acerr': [0.12536650169770894, 0.09107895692161581, 0.5213665660745976, 0.000470663333955102, 355.8505636471652]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img163.fz': {
        'os': [-0.27603895527298167, 60.28720015638354, 88881.8621480748],
        'oserr': [0.054275802986569276, 0.0395327175871201, 79.53603171315038],
        'ac': [-11.070384811532074, 65.33998874528783, 399.21378440491975, 0.061283403820100814, 194037.09368120442],
        'acerr': [0.14405544941206994, 0.10495556696023603, 0.5923821797578306, 0.0005476689836020455, 404.03076649358667]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img157.fz': {
        'os': [ -0.018, 59.636, 87564.396],
        'ac': [ -10.59, 65.059, 399.233, 0.059, 195685.933]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img154.fz': {
        'os': [ -0.172, 59.655, 87554.441],
        'ac': [ -11.456, 65.035, 401.517, 0.065, 194096.201]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img151.fz': {
        'os': [ -0.517, 59.194, 87434.028],
        'ac': [ -11.66, 64.927, 396.571, 0.064, 193951.027]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img148.fz': {
        'os': [ -0.341, 59.51, 87419.442],
        'ac': [ -10.466, 64.838, 398.367, 0.066, 193970.822]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img145.fz': {
        'os': [ 0.01, 59.333, 87386.978],
        'ac': [ -11.263, 64.719, 397.453, 0.066, 195231.348]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img142.fz': {
        'os': [ -0.483, 59.68, 87833.176],
        'ac': [ -11.355, 65.077, 399.757, 0.071, 193698.868],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img139.fz': {
        'os': [ -0.005, 59.279, 87535.592],
        'ac': [ -11.301, 64.93, 397.061, 0.074, 194410.147],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img136.fz': {
        'os': [ -0.266, 59.745, 87840.56 ],
        'ac': [ -11.566, 65.156, 399.143, 0.076, 193081.738],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img133.fz': {
        'os': [ -0.164, 59.665, 87822.324],
        'ac': [ -11.092, 65.334, 411.001, 0.078, 193198.059],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img130.fz': {
        'os': [ -0.481, 59.385, 87444.874],
        'ac': [ -10.562, 65.021, 399.202, 0.081, 194859.854],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img127.fz': {
        'os': [ -0.294, 59.192, 87481.826],
        'ac': [ -11.343, 65.142, 400.385, 0.085, 194955.314],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img124.fz': {
        'os': [ -0.425, 59.721, 87232.754],
        'ac': [ -11.132, 65.325, 400.733, 0.085, 194298.829],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img121.fz': {
        'os': [ -0.392, 59.651, 87438.021],
        'ac': [ -10.755, 65.488, 400.629, 0.082, 194875.764],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img118.fz': {
        'os': [ -0.51, 59.641, 87376.276],
        'ac': [ -11.368, 65.436, 401.484, 0.083, 194846.273]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img115.fz': {
        'os': [ -0.252, 60.584, 87743.606],
        'ac': [ -11.068, 66.005, 403.335, 0.088, 194146.206],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img112.fz': {
        'os': [ -0.273, 59.058, 87484.541],
        'ac': [ -10.895, 64.738, 401.384, 0.082, 194120.448],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img109.fz': {
        'os': [ -0.253, 59.093, 87497.08 ],
        'ac': [ -11.507, 64.761, 399.258, 0.085, 194482.02 ]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img106.fz': {
        'os': [ -0.129, 59.317, 87671.988],
        'ac': [ -11.108, 65.126, 396.289, 0.087, 193857.804]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img103.fz': {
        'os': [ 0.052, 58.997, 87437.662],
        'ac': [ -11.121, 64.922, 398.255, 0.083, 194618.358],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img100.fz': {
        'os': [ -0.293, 59.247, 87407.631],
        'ac': [ -11.191, 64.817, 396.737, 0.085, 195086.393],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img97.fz': {
        'os': [ -0.045, 59.898, 87534.062],
        'ac': [ -11.116, 65.605, 399.416, 0.085, 195100.49 ],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img94.fz': {
        'os': [ -0.297, 60.813, 87404.554],
        'ac': [ -10.749, 66.788, 401.777, 0.087, 194628.093],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img91.fz': {
        'os': [ -0.231, 62.059, 87401.257],
        'ac': [ -11.019, 67.621, 398.842, 0.086, 195249.606],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img88.fz': {
        'os': [ -0.306, 61.009, 87479.733],
        'ac': [ -10.94, 66.654, 400.212, 0.086, 195821.729],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img85.fz': {
        'os': [ -0.249, 58.969, 87408.985],
        'ac': [ -10.828, 64.984, 397.614, 0.088, 195106.664],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img82.fz': {
        'os': [ -0.154, 59.085, 87438.336],
        'ac': [ -11.052, 64.934, 399.838, 0.092, 194125.971],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img79.fz': {
        'os': [ -0.13, 59.105, 87457.792],
        'ac': [ -10.677, 64.644, 399.079, 0.09, 195090.389],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img76.fz': {
        'os': [ -0.11, 58.956, 87519.72 ],
        'ac': [ -10.96, 64.889, 395.746, 0.095, 195046.987],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img73.fz': {
        'os': [ -0.005, 58.951, 87522.034],
        'ac': [ -11.065, 64.947, 401.609, 0.095, 195250.731],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img70.fz': {
        'os': [ -0.198, 59.872, 87862.163],
        'ac': [ 386.326, 67.585, 402.993, 0.09, 19290.243],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img67.fz': {
        'os': [ -0.177, 59.333, 87423.912],
        'ac': [ -10.628, 64.997, 399.421, 0.105, 194309.604],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img64.fz': {
        'os': [ -0.264, 59.341, 87328.298],
        'ac': [ -11.537, 64.684, 398.197, 0.111, 194660.964],
    }
}


arr = np.array( 
    [ (
        int(re.search("img([0-9]+)\.fz", path).groups()[0]), 
        data['os'][1], 
        data['ac'][1], 
        data['ac'][3], 
        data['ac'][2] 
    )
    for path, data in evo.items() ] 
)
# print(arr)

fig, axes = plt.subplots(2)
axes[0].plot( arr[:,0], arr[:,1]/arr[:,4], '.', label = 'os noise' )
axes[0].plot( arr[:,0], arr[:,2]/arr[:,4], '.', label = 'ac noise' )
axes[0].set_ylabel(r'$\sigma$ [e-]')
axes[0].grid(True)
axes[0].legend(frameon=False)
axes[1].plot( arr[:,0], arr[:,3] * 24/1.5, '.', label = 'dark current')
axes[1].set_ylabel(r'$\lambda$ [e-/pix/day]')
axes[1].legend(frameon=False)
axes[1].grid(True)
plt.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …